In [89]:
from fuzzywuzzy import fuzz, process

In [90]:
import json

In [91]:
from transit_parser import *

In [92]:
stops_df = pd.read_csv(RAIL_DATA + '/stops.txt')

In [93]:
stops_df = stops_df[~stops_df['stop_name'].str.contains("LIGHT RAIL")]

In [94]:
stops_dict = stops_df[['stop_id', 'stop_name']].set_index('stop_name').T.to_dict('list')

In [95]:
stops_dict = dict((k.lower(), v) for k,v in stops_dict.iteritems())

In [96]:
stops_list = list(stops_df['stop_name'])

In [97]:
stops_list = [s.lower() for s in stops_list]

In [98]:
t = TrainParser('./scraped_data/2018_02_09/2018_02_09_3961')

In [99]:
t_stops = t.get_stop_times()

In [100]:
t_rows = t.get_rows(t_stops)

In [101]:
t_stops

[{'station': u'New York Penn Station',
  'status': 'Departed',
  'time': u'2018-02-09 18:39:07.970835'},
 {'station': u'Secaucus Concourse',
  'status': 'Departed',
  'time': u'2018-02-09 18:47:05.137277'},
 {'station': u'Newark Penn Station',
  'status': 'Departed',
  'time': u'2018-02-09 19:05:16.479151'},
 {'station': u'Newark Airport',
  'status': 'Departed',
  'time': u'2018-02-09 19:17:07.282859'},
 {'station': u'New Brunswick',
  'status': 'Departed',
  'time': u'2018-02-09 19:29:05.788174'},
 {'station': u'Jersey Avenue',
  'status': 'Departed',
  'time': u'2018-02-09 19:33:08.486474'},
 {'station': u'Princeton Junction',
  'status': 'Departed',
  'time': u'2018-02-09 19:47:05.412448'},
 {'station': u'Hamilton',
  'status': 'Departed',
  'time': u'2018-02-09 19:54:03.523446'},
 {'station': u'Trenton', 'status': 'Departed', 'time': '2018-02-09 20:05:00'}]

In [102]:
stops_dict

{'aberdeen-matawan': [37169],
 'absecon': [2],
 'allendale': [3],
 'allenhurst': [4],
 'anderson street': [5],
 'annandale': [6],
 'asbury park': [8],
 'atco': [9],
 'atlantic city': [10],
 'avenel': [11],
 'basking ridge': [12],
 'bay head': [13],
 'bay street': [14],
 'belmar': [15],
 'bergenline ave': [38578],
 'berkeley heights': [17],
 'bernardsville': [18],
 'bloomfield': [19],
 'boonton': [20],
 'bound brook': [21],
 'bradley beach': [22],
 'brick church': [23],
 'bridgewater': [24],
 'broadway': [25],
 'campbell hall': [26],
 'chatham': [27],
 'cherry hill': [28],
 'clifton': [29],
 'convent': [30],
 'cranford': [32],
 'delawanna': [33],
 'denville': [34],
 'dover': [35],
 'dunellen': [36],
 'east orange': [37],
 'edison station': [38],
 'egg harbor': [39],
 'elberon': [40],
 'elizabeth': [41],
 'emerson': [42],
 'essex street': [43],
 'fanwood': [44],
 'far hills': [45],
 'garfield': [46],
 'garwood': [47],
 'gillette': [48],
 'gladstone': [49],
 'glen ridge': [50],
 'glen roc

In [129]:
def get_station_match(station):
    match = process.extract(station.lower(), stops_list, limit=1, scorer=fuzz.ratio)
    if match[0][1] > 77:
        return match
    else:
        match = process.extract(station.lower(), stops_list, limit=1, scorer=fuzz.partial_ratio)
    if match[0][1] > 77:
        return match[0][0]
    return None

In [130]:
get_station_match("princeton junction")

[('princeton junction', 100)]

In [132]:
test = {}
for station in ALL_STATIONS:
    match = get_station_match(station)
    if match is not None:
        test[station] = stops_dict[match[0][0]][0]
    else:
        test[station] = None

In [133]:
test

{'Aberdeen-Matawan': 37169,
 'Absecon': 2,
 'Allendale': 3,
 'Allenhurst': 4,
 'Anderson Street': 5,
 'Annandale': 6,
 'Asbury Park': 8,
 'Atco': 9,
 'Atlantic City Rail Terminal': 10,
 'Avenel': 11,
 'Basking Ridge': 12,
 'Bay Head': 13,
 'Bay Street': 14,
 'Belmar': 15,
 'Berkeley Heights': 17,
 'Bernardsville': 18,
 'Bloomfield': 19,
 'Boonton': 20,
 'Bound Brook': 21,
 'Bradley Beach': 22,
 'Brick Church': 23,
 'Bridgewater': 24,
 'Broadway Fair Lawn': 25,
 'Campbell Hall': 26,
 'Chatham': 27,
 'Cherry Hill': 28,
 'Clifton': 29,
 'Convent Station': 30,
 'Cranford': 32,
 'Delawanna': 33,
 'Denville': 34,
 'Dover': 35,
 'Dunellen': 36,
 'East Orange': 37,
 'Edison': 38,
 'Egg Harbor City': 39,
 'Elberon': 40,
 'Elizabeth': 41,
 'Emerson': 42,
 'Essex Street': 43,
 'Fanwood': 44,
 'Far Hills': 45,
 'Garfield': 46,
 'Garwood': 47,
 'Gillette': 48,
 'Gladstone': 49,
 'Glen Ridge': 50,
 'Glen Rock Boro Hall': 51,
 'Glen Rock Main Line': 52,
 'Hackettstown': 54,
 'Hamilton': 32905,
 'Hamm

In [134]:
with open('rail_data/rail_stations', 'w') as outfile:
    json.dump(test, outfile)